In [16]:
import queue

import dataclasses
import typing

class CallbackQueue:
    @dataclasses.dataclass
    class Task:
        func: typing.Callable | None = None
        result: typing.Any = None

    def __init__(self, maxsize=1):
        self.base = queue.Queue(maxsize=maxsize)

    def __call__(self, *args, **kwargs):
        task: self.Task = self.base.get()
        if task.func is not None:
            task.result = task.func(*args, **kwargs)
        self.base.task_done()

    def call(self, func: typing.Callable | None = None):
        task = self.Task(func=func)
        self.base.put(task)
        self.base.join()
        return task.result

class CloseableCallbackQueue(CallbackQueue):
    class Closed(Exception):
        pass

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.closed = False

    def __call__(self, *args, **kwargs):
        if self.closed:
            raise self.Closed()
        return super().__call__(*args, **kwargs)
    
    def call(self, func: typing.Callable):
        if self.closed:
            raise self.Closed()
        return super().call(func)
    
    def close(self):
        self.closed = True
        # NOTE ref https://stackoverflow.com/a/18873213
        while not self.base.empty():
            try: self.base.get(block=False)
            except queue.Empty: continue
            self.base.task_done()

In [24]:
import energyplus.ooep.ems


import contextlib

class BaseStateMachine:
    class StepFunction:
        def __init__(
            self, 
            state_machine: 'BaseStateMachine',
            event_specs: energyplus.ooep.ems.Environment.Event.Specs
        ):
            self._callback = CloseableCallbackQueue()
            self._state_machine = state_machine
            self._event_specs = event_specs
        
        @contextlib.contextmanager
        def __state_context__(self):
            self._state_machine._env.event_listener.subscribe(
                self._event_specs,
                self._callback,
            )
            yield
            self._callback.close()

        def __call__(self, func: typing.Callable | None = None):
            return self._callback.call(func)

    def __init__(
        self, 
        env: energyplus.ooep.ems.Environment
    ):
        self._env = env
        self._step_funcs: typing.List[self.StepFunction] = []

    def step_function(self, event_specs):
        f = self.StepFunction(self, event_specs)
        self._step_funcs.append(f)
        return f

    def run_blocking(self, *args, **kwargs):
        with contextlib.ExitStack() as stack:
            for f in self._step_funcs:
                stack.enter_context(f.__state_context__())
            return self._env(*args, **kwargs)


import threading

class StateMachine(BaseStateMachine):
    def run(self, *args, **kwargs):
        thr = threading.Thread(
            target=self.run_blocking, 
            args=args, kwargs=kwargs,
        )
        thr.start()


In [25]:
from energyplus.dataset.basic import dataset as epds

env = energyplus.ooep.ems.Environment().__enter__()

sm_env = StateMachine(env)

sm_stepf = sm_env.step_function(
    dict(event_name='begin_new_environment')
)

sm_env.run(
    # TODO
    '--design-day',
    #'--annual',
    '--output-directory', 'build/demo-eplus',
    '--weather', f'{epds.weathers}/USA_FL_Tampa.Intl.AP.722110_TMY3.epw',
    f'{epds.models}/ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf',
    verbose=True,
)

EnergyPlus Starting
EnergyPlus, Version 23.2.0-7636e6b3e9, YMD=2023.12.26 22:39


Initializing Response Factors
Calculating CTFs for "INTERIORFURNISHINGS"
Calculating CTFs for "DROPCEILING"
Calculating CTFs for "INT_WALL"
Calculating CTFs for "EXT_SLAB_8IN_WITH_CARPET"
Calculating CTFs for "INT_SLAB_FLOOR"
Calculating CTFs for "NONRES_ROOF"
Calculating CTFs for "NONRES_EXT_WALL"
Calculating CTFs for "BASEMENT_WALL_EAST_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_SOUTH_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_SOUTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_WEST_CFACTOR"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing S

In [26]:
sm_stepf()

Warming up {1}
Calculating Detailed Daylighting Factors, Start Date=12/21


Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Warming up {7}
Warming up {8}
Starting Simulation at 12/21 for DENVER-AURORA-BUCKLEY.AFB_CO_USA ANN HTG 99.6% CONDNS DB
Initializing New Environment Parameters


In [ ]:
# TODO callback
# TODO func step(action: callable): wait for `action` to be performed; ret observations;
# TODO get; get obs; task_done